# Configure MLflow 🧐

In [3]:
import requests
from getpass import getpass
import datetime

**Set Environment Variables**


**Initialize MLflow**

In [4]:
!pip install mlflow --quiet

import mlflow
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


**Set Local Configurations**

Under the [Token tab](https://dagshub.com/user/settings/tokens) in the user setting, copy the default token and use it here.

### Source
- https://web.stanford.edu/~jurafsky/slp3/3.pdf
- https://huyenchip.com/2023/05/02/rlhf.html#language_model

In [5]:
# REPO_OWNER = input('Repo owner: ').strip()
# REPO_NAME = input('Repo name: ').strip()


REPO_OWNER = 'lnnchau'
REPO_NAME = 'mlops-lyrics-lm'
USER_NAME = REPO_OWNER

os.environ['MLFLOW_TRACKING_USERNAME'] = USER_NAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Enter your DAGsHub access token: ')

mlflow.set_tracking_uri(f'https://dagshub.com/{REPO_OWNER}/{REPO_NAME}.mlflow')

Enter your DAGsHub access token: ··········


In [6]:
import mlflow

In [7]:
model = mlflow.pytorch.load_model('models:/BigramLanguageModel/5', map_location='cpu')
import torch
context = torch.zeros((1, 1), dtype=torch.long)


model(context)

(tensor([[[ 3.1897, -0.1859, -2.3211, -1.6164, -3.7888, -1.3246, -2.0342,
           -0.1595, -3.3363, -0.2134,  0.7939, -3.1611, -3.9309, -4.2971,
           -2.1758, -2.5180,  4.0481,  2.7661, -4.5270, -2.5663, -0.1953,
           -1.5023,  2.7343,  0.6043,  3.5420, -2.7502,  2.0626, -1.0538,
            0.6115, -1.6857, -3.4115, -2.4753, -3.5058, -3.4910, -3.0509,
           -4.1252,  0.2435, -1.4038, -0.8380, -1.8344,  0.7848, -0.7693,
           -0.2874, -2.8874,  3.3461, -4.0209, -1.3799, -1.3985, -0.0450,
           -0.8859, -2.1166]]], grad_fn=<ViewBackward0>),
 None)

## Language model
- GPT is a LLM (large language models)
- Language model encodes the statistical information of the language. It tells you what's likely to appear in a context.
    - Example: (find 2 examples predicting next word / fill in the blanks)
    - To train a language model, you feed it a lot of text (training data) so that it can learn the statistical information from it.
- Word-level vs Character-level?
- This notebook: Character-level


## n-gram model
### What this model do?
- Predict how likely the next word is, given n-1 preceding words in the sequence.
- For example,
    - n=2 (bigram model)
    - assume our dictionary has the following words: to, I, movies, like, watch, you, we, books
    - input: `I like to watch ____`
    - context sequence: `watch`
    - what the bigram does, is that, it'll go over all words in the dictionary, and compute how likely it is the next word - more formally, `P(word_i|watch)`

### How to train this model?
yadayadayada

#### Build the vocabulary
- Some concepts in NLP
- `Document`: text objects, which could be an article, a movie review, a passage or even a sentence.
- `Corpus`: list of documents
- `Vocabulary`: list of all the tokens in all documents. based on the task, token could be either a word, a character, or parts of the word (e.g. `playing` can be split into two tokens `play` and `ing`)

In [8]:
import pandas as pd

In [9]:
data = pd.read_csv('/content/drive/MyDrive/spotify_millsongdata.csv')
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [10]:
corpus = data.text.str.lower()
sample_document = corpus[0]

print(f'Corpus has {len(corpus)} documents')
print(f'Sample document: {sample_document}')

Corpus has 57650 documents
Sample document: look at her face, it's a wonderful face  
and it means something special to me  
look at the way that she smiles when she sees me  
how lucky can one fellow be?  
  
she's just my kind of girl, she makes me feel fine  
who could ever believe that she could be mine?  
she's just my kind of girl, without her i'm blue  
and if she ever leaves me what could i do, what could i do?  
  
and when we go for a walk in the park  
and she holds me and squeezes my hand  
we'll go on walking for hours and talking  
about all the things that we plan  
  
she's just my kind of girl, she makes me feel fine  
who could ever believe that she could be mine?  
she's just my kind of girl, without her i'm blue  
and if she ever leaves me what could i do, what could i do?




In [11]:
corpus_as_string = ' '.join(corpus.values)
vocab = set(corpus_as_string)
vocab_size = len(vocab)

print(f'Length of vocab: {vocab_size}')
print(sorted(list(vocab)))

Length of vocab: 51
['\n', '\r', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
class Tokenizer:
    def __init__(self, vocab):
        # create a mapping from characters to integers
        self.stoi = { ch:i for i,ch in enumerate(vocab) }
        self.itos = { i:ch for i,ch in enumerate(vocab) }

    def encode(self, s):
        # encoder: take a string, output a list of integers
        return [self.stoi[c] for c in s]

    def decode(self, l):
        # decoder: take a list of integers, output a string
        return ''.join([self.itos[i] for i in l])

tokenizer = Tokenizer(vocab)
print(tokenizer.encode("hii there"))
print(tokenizer.decode(tokenizer.encode("hii there")))

[3, 48, 48, 40, 41, 3, 43, 16, 43]
hii there


In [13]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[3, 48, 48, 40, 41, 3, 43, 16, 43]
hii there


In [14]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(corpus_as_string), dtype=torch.long)
print(data.shape, data.dtype)
# print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([70426172]) torch.int64


#### Create dataset
- train_size: 90%
- val_size: 105
- seq_len (block size for now): 8
    - what is the maximum context length for predictions?
- batch_size = 4 # how many independent sequences will we process in parallel?


In [15]:
# Let's now split up the data into train and validation sets
n = int(0.8*len(data)) # first 90% will be train, rest val

train_ratio, val_ratio, test_ratio = 0.9, 0.05, 0.05

train_size = int(train_ratio * len(data))
val_size = int(val_ratio * len(data))
test_size = int(test_ratio * len(data))

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[-test_size:]

assert len(train_data) == train_size
assert len(val_data) == val_size
assert len(test_data) == test_size

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([15, 37, 37, 47, 40, 13, 41, 40,  3])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([15]) the target: 37
when input is tensor([15, 37]) the target: 37
when input is tensor([15, 37, 37]) the target: 47
when input is tensor([15, 37, 37, 47]) the target: 40
when input is tensor([15, 37, 37, 47, 40]) the target: 13
when input is tensor([15, 37, 37, 47, 40, 13]) the target: 41
when input is tensor([15, 37, 37, 47, 40, 13, 41]) the target: 40
when input is tensor([15, 37, 37, 47, 40, 13, 41, 40]) the target: 3


In [18]:

torch.manual_seed(1337)

def get_batch(split, batch_size, block_size):
    '''
    Generate a small batch of data of inputs x and targets y

    return: x, y
        - x: (batch_size, block_size)
        - y: (batch_size, block_size)
    '''
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

get_batch('train', 4, 8)


(tensor([[27, 43, 36, 40, 28, 37, 41, 40],
         [29, 40, 30, 13, 28, 48, 27, 40],
         [43, 40, 23, 13, 15, 15, 29, 40],
         [40, 18, 37, 34, 16, 40, 13, 16]]),
 tensor([[43, 36, 40, 28, 37, 41, 40, 41],
         [40, 30, 13, 28, 48, 27, 40, 20],
         [40, 23, 13, 15, 15, 29, 40, 40],
         [18, 37, 34, 16, 40, 13, 16, 30]]))

## Modeling

## Interface



### `Config`
Store parameters settings

In [ ]:
class Config:
    def __init__(self,
                 batch_size,
                 max_iterations,
                 eval_iterations,
                 eval_intervals,
                 lr,
                 vocab_size,
                 block_size,
                 n_embd,
                 n_head,
                 n_layer,
                 dropout):
        self.batch_size = batch_size
        self.max_iterations = max_iterations
        self.eval_iterations = eval_iterations
        self.eval_intervals = eval_intervals

        self.lr = lr
        self.vocab_size = vocab_size
        self.block_size = block_size

        self.n_embd = n_embd
        self.n_head = n_head
        self.n_layer = n_layer
        self.dropout = dropout

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'


### ML architectures
Should receive `Config` object as param to construction method


#### Model 1: Simple BigramLanguageModel

In [19]:
from IPython.core.formatters import Configurable
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, config: Configurable):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        vocab_size = config.vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (batch_size,seq_len) tensor of integers

        logits = self.token_embedding_table(idx) # (batch_size,seq_len,vocab_size)
        
        if targets is None:
            loss = None
        else:
            batch_size, seq_len, vocab_size = logits.shape

            # example: we have 2 classes [0, 1]
            # logits = [[0.5, 0.5], [0.3, 0.7], [0.6, 0.4]]
            # targets = [0, 1, 0]

            logits = logits.view(batch_size*seq_len, vocab_size)
            targets = targets.view(batch_size*seq_len)
  
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (batch_size, vocab_size)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (batch_size, vocab_size)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (batch_size, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    @torch.no_grad()
    def evaluate_ppl(self, seq_tensor):
        # Implemented from https://web.stanford.edu/~jurafsky/slp3/3.pdf
        logits, _ = self(seq_tensor)    # logits = (batch_size, seq_len, vocab_size)

        batch_size, seq_len, vocab_size = logits.shape

        logits = logits.view(batch_size*seq_len, vocab_size)
        
        logits = logits[:-1, :] # to compute P(x_i|x_(i-1))
        probs = F.softmax(logits, dim=-1)
        
        ground_truths = seq_tensor.view(batch_size*seq_len)[1:]
        ppl = probs[
            torch.arange(batch_size*seq_len - 1),
            ground_truths]

        return torch.exp(ppl.mean())

#### Model 2: Bigram Language Model with Attention

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, config: Config):
        super().__init__()

        n_embd = config.n_embd
        block_size = config.block_size
        dropout = config.dropout

        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, config: Config):
        super().__init__()
    
        n_head = config.n_head
        n_embd = config.n_embd
        dropout = config.dropout

        head_size = n_embd // n_head

        self.heads = nn.ModuleList([Head(head_size, config) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd, dropout):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, config: Config):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()

        n_embd = config.n_embd
        n_head = config.n_head
        dropout = config.dropout
        
        self.sa = MultiHeadAttention(config)
        self.ffwd = FeedFoward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModelAttention(nn.Module):

    def __init__(self, config: Config):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(config.vocab_size, config.n_embd)
        self.position_embedding_table = nn.Embedding(config.block_size, config.n_embd)

        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])

        self.ln_f = nn.LayerNorm(config.n_embd) # final layer norm
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size)

    @torch.no_grad()
    def evaluate_ppl(self, seq_tensor):
        _, test_size = seq_tensor.shape
        num_chunks = test_size // block_size

        # update test size to fit with block_size
        test_size = num_chunks * block_size
        seq_tensor = seq_tensor[:, :test_size]

        # split seq_tensor into chunks of blocksize
        test_input = seq_tensor.view(num_chunks, block_size)

        # Implemented from https://web.stanford.edu/~jurafsky/slp3/3.pdf
        logits, _ = self(test_input)    # logits = (batch_size, seq_len, vocab_size)

        batch_size, seq_len, vocab_size = logits.shape

        logits = logits.view(test_size, vocab_size)
        logits = logits[:-1, :] # to compute P(x_i|x_(i-1))
        probs = F.softmax(logits, dim=-1)
        
        # TODO: write docs to explain this part
        ground_truths = test_input.view(test_size)[1:]
        ppl = probs[
            torch.arange(test_size - 1),
            ground_truths]

        return torch.exp(ppl.mean())

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


### `LyricsGenerator`
Wrapper for inference - should receive `model` and `tokenizer` as params to contruction method

In [20]:
class LyricsGenerator:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_lyrics(self, start_phrase, max_new_tokens=2000):
        start_phrase_as_ids = self.tokenizer.encode(start_phrase)
        context = torch.tensor(start_phrase_as_ids, dtype=torch.long, device=self.device).reshape(1, -1)
        output_tokens = self.model.generate(idx=context, max_new_tokens=max_new_tokens)[0].tolist()

        return self.tokenizer.decode(output_tokens)

### `Trainer`
Encapsulates training and evaluate code

In [70]:
import time

In [75]:
class Trainer:
    def __init__(self, model_class, config, get_batch):
        self.config = config
        self.model_class = model_class
        self.model = model_class(config)
        self.get_batch = get_batch

    @torch.no_grad()
    def estimate_loss(self):
        model = self.model
        config = self.config

        out = {}
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(config.eval_iterations)
            for k in range(config.eval_iterations):
                X, Y = self.get_batch(split, config.batch_size, config.block_size)
                X, Y = X.to(config.device), Y.to(config.device)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        model.train()
        return out

    def evaluate_test(self, test_data):
        device = self.config.device
        model = self.model

        model.eval()

        model = model.to(device)
        test_data = test_data.to(device)
        ppl = model.evaluate_ppl(test_data.view(1, -1))

        mlflow.log_metric('ppl', ppl)
        return ppl


    def train(self):
        model_class = self.model_class
        config = self.config

        mlflow.set_experiment_tag('architecture', model_class.__name__)
        mlflow.log_params(config.__dict__)

        m = self.model
        m.to(config.device)

        optimizer = torch.optim.AdamW(m.parameters(), lr=config.lr)

        m.train()

        for step in range(config.max_iterations):
            if step % config.eval_intervals == 0 or step == config.max_iterations - 1:
                losses = self.estimate_loss()
                print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
                mlflow.log_metrics(losses, step=step)

            # sample a batch of data
            xb, yb = self.get_batch(
                'train',
                batch_size=config.batch_size,
                block_size=config.block_size)
            xb, yb = xb.to(config.device), yb.to(config.device)

            # evaluate the loss
            logits, loss = m(xb, yb)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

        m.to("cpu")
        mlflow.pytorch.log_model(m, "model")

        return m

## Run experiments

In [72]:
import pickle
import time

In [78]:
config = Config(
    batch_size = 16, # how many independent sequences will we process in parallel?
    block_size = 32, # what is the maximum context length for predictions?
    max_iterations = 10000,
    eval_intervals = 100,
    lr = 1e-3,
    eval_iterations = 200,
    n_embd = 64,
    n_head = 4,
    n_layer = 4,
    dropout = 0.0,
    vocab_size=vocab_size,
)

In [79]:
torch.cuda.empty_cache()

In [ ]:
test_data = data[-test_size:]

tokenizer = Tokenizer(vocab)
tokenizer_path = 'tokenizer.pkl'

with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

trainer = Trainer(
    BigramLanguageModelAttention,
    config=config,
    get_batch=get_batch
)

with mlflow.start_run(run_name=str(time.time())):
    trainer.train()
    trainer.evaluate_test(test_data)

    mlflow.log_artifact(tokenizer_path, "model/artifacts")

step 0: train loss 4.1192, val loss 4.1194
step 100: train loss 2.4006, val loss 2.3954
step 200: train loss 2.2343, val loss 2.2241
step 300: train loss 2.1417, val loss 2.1217
step 400: train loss 2.0808, val loss 2.0735
step 500: train loss 2.0126, val loss 2.0015
step 600: train loss 1.9557, val loss 1.9365
step 700: train loss 1.9240, val loss 1.9025
step 800: train loss 1.8797, val loss 1.8709
step 900: train loss 1.8415, val loss 1.8349
step 1000: train loss 1.8033, val loss 1.7939
step 1100: train loss 1.7751, val loss 1.7766
step 1200: train loss 1.7570, val loss 1.7446
step 1300: train loss 1.7381, val loss 1.7135
step 1400: train loss 1.7216, val loss 1.7103
step 1500: train loss 1.7069, val loss 1.6974
step 1600: train loss 1.7052, val loss 1.6971
step 1700: train loss 1.6739, val loss 1.6619
step 1800: train loss 1.6763, val loss 1.6614
step 1900: train loss 1.6522, val loss 1.6399
step 2000: train loss 1.6462, val loss 1.6408
step 2100: train loss 1.6384, val loss 1.6234
